<a href="https://colab.research.google.com/github/jtanzi56/Prediction-of-Product-Sales/blob/main/Sales_Prediction_jtanzi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Prediction of Product Sales
Jacob Tanzi



##Project Overview


##Load and Inspect Data


##Clean Data


##Exploratory Data Analysis


##Explanatory Data Analysis